<a href="https://colab.research.google.com/github/sanbgos/Se-alesysistemas/blob/main/DashBoardmodulacionAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit yt-dlp pydub librosa ffmpeg-python pyngrok --quiet


In [ ]:
%%writefile app.py
import streamlit as st
import yt_dlp
from pydub import AudioSegment
import librosa
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq
import soundfile as sf
import os

# Función: Descargar y procesar audio
def procesar_audio(url):
    output = "cancion_descargada"
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output + '.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }],
        'quiet': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    audio = AudioSegment.from_file(output + ".mp3")
    fragmento = audio[20_000:25_000]
    fragmento.export("mensaje.wav", format="wav")

    mensaje, sr = librosa.load("mensaje.wav", sr=None)
    t = np.linspace(0, len(mensaje)/sr, len(mensaje))
    mensaje = mensaje / np.max(np.abs(mensaje))
    return mensaje, sr, t

# Filtro ideal pasa bajas
def filtro_ideal_pasabajas(signal, fs, fc):
    N = len(signal)
    freq = np.fft.rfftfreq(N, d=1/fs)
    espectro = np.fft.rfft(signal)
    filtro = freq <= fc
    espectro_filtrado = espectro * filtro
    señal_filtrada = np.fft.irfft(espectro_filtrado, n=N)
    return señal_filtrada

# Gráfica espectral
def graficar_fft(signal, sr, title):
    N = len(signal)
    fft_vals = np.abs(rfft(signal)) / N
    freqs = rfftfreq(N, 1/sr)
    plt.plot(freqs, fft_vals)
    plt.title(f"Espectro de {title}")
    plt.xlabel("Frecuencia [Hz]")
    plt.ylabel("Magnitud")

# Guardar señal como WAV
def guardar_wav(nombre, signal, sr):
    sf.write(nombre, signal, sr)

# Streamlit App
st.title("🎙️ Dashboard: Modulación y Demodulación AM")

url = st.text_input("Ingresa el enlace de YouTube:", value="https://www.youtube.com/watch?v=0ElD2qJ5ZoU")

if st.button("Procesar audio"):
    with st.spinner("Descargando y procesando..."):
        mensaje, sr, t = procesar_audio(url)

        Ac = 1.0
        fc = 10000
        portadora = Ac * np.cos(2 * np.pi * fc * t)
        modulada = (1 + mensaje / Ac) * portadora

        mezcla = modulada * portadora
        recuperado_crudo = filtro_ideal_pasabajas(mezcla, sr, fc=4000)
        recuperado = 2 * recuperado_crudo

        # Guardar audios
        guardar_wav("portadora.wav", portadora, sr)
        guardar_wav("modulada.wav", modulada, sr)
        guardar_wav("mezcla.wav", mezcla, sr)
        guardar_wav("recuperado.wav", recuperado, sr)

        # Mostrar resultados en orden
        st.markdown("### 🎧 1. Audio Mensaje Original")
        st.audio("mensaje.wav", format="audio/wav")

        st.markdown("### 🎧 2. Portadora (sin información)")
        st.audio("portadora.wav", format="audio/wav")

        st.markdown("### 🎧 3. Señal AM Modulada")
        st.audio("modulada.wav", format="audio/wav")

        st.markdown("### 🎧 4. Señal Mezclada (tras multiplicar por la portadora)")
        st.audio("mezcla.wav", format="audio/wav")

        st.markdown("### 🎧 5. Mensaje Recuperado (después del filtro)")
        st.audio("recuperado.wav", format="audio/wav")

        # Gráficas en orden
        fig1, axs1 = plt.subplots(3, 1, figsize=(10, 8))
        axs1[0].plot(t, mensaje)
        axs1[0].set_title("Mensaje original (tiempo)")
        axs1[1].plot(t[:int(sr*0.002)], portadora[:int(sr*0.002)])
        axs1[1].set_title("Portadora (zoom 2 ms)")
        axs1[2].plot(t[:int(sr*0.002)], modulada[:int(sr*0.002)])
        axs1[2].set_title("Modulada (zoom 2 ms)")
        plt.tight_layout()
        st.pyplot(fig1)

        fig2 = plt.figure(figsize=(10, 8))
        plt.subplot(3,1,1)
        graficar_fft(mensaje, sr, "mensaje")
        plt.subplot(3,1,2)
        graficar_fft(portadora, sr, "portadora")
        plt.subplot(3,1,3)
        graficar_fft(modulada, sr, "modulada")
        plt.tight_layout()
        st.pyplot(fig2)

        fig3, axs3 = plt.subplots(3, 1, figsize=(10, 8))
        axs3[0].plot(t, mensaje)
        axs3[0].set_title("Mensaje original (tiempo)")
        axs3[1].plot(t, mezcla)
        axs3[1].set_title("Señal mezclada (tiempo)")
        axs3[2].plot(t, recuperado)
        axs3[2].set_title("Mensaje recuperado (tiempo)")
        plt.tight_layout()
        st.pyplot(fig3)


In [ ]:
from pyngrok import ngrok
import os
import threading

# Token de Ngrok ya incluido
NGROK_AUTH_TOKEN = "2yZZCOImGCr3tXOrM87zpgVub0E_3V3vvZwdauL41ZB8xHHck"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Ejecutar Streamlit en segundo plano
def run():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=run)
thread.start()

# Túnel público
public_url = ngrok.connect(8501)
print("🔗 Tu app está en:", public_url)